In [9]:
from sklearn.model_selection import train_test_split

# X = [1,2,3,4]
# y = [1,2,3,4]

train_X, test_X, train_y, test_y = train_test_split(X, y, shuffle=True, test_size = 0.2, random_state = 10)

In [7]:
# Linear Regression
from sklearn.linear_model import LinearRegression 
reg = LinearRegression().fit(train_X, train_y) 
reg.score(train_X, train_y) 
reg.coef_ 
reg.intercept_ 
reg.predict(test_X) 

from sklearn.linear_model import Ridge 
rid = Ridge(alpha=1.0).fit(X, y)
from sklearn.linear_model import Lasso 
las = Lasso(alpha=1.0).fit(X, y)
from sklearn.linear_model import ElasticNet 
ela = ElasticNet(l1_ratio=0.5).fit(X, y)


In [ ]:
#Logistic Regression 
from sklearn.linear_model import LogisticRegression 
lgr = LogsticRegression().fit(train_X, train_y) 
lgr.Score(train_X, train_y)
lgr.predict(test_X) 
lgr.predict_proba(test_X) 

In [ ]:
# Decision Tree Ensemble

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=10, max_depth=2).fit(train_X, train_y)
from sklearn.ensemble import RandomForestRegressor
rfc = RandomForestRegressor(max_depth=2, n_estimators=10).fit(train_X, train_y)

from sklearn.ensemble import AdaBoostClassifier
adb = AdaBoostClassifier(n_estimators=10, max_depth=2).fit(train_X, train_y)
from sklearn.ensemble import AdaBoostRegressor
adb = AdaBoostRegressor(DecisionTreeRegressor(max_depth=2), n_estimators=10).fit(train_X, train_y)

from sklearn.ensemble import GradientBoostingClassifier
grb = GradientBoostingClassifier(n_estimators=10, max_depth=2).fit(train_X, train_y)
from sklearn.ensemble import GradientBoostingRegressor
grb = GradientBoostingRegressor(DecisionTreeRegressor(max_depth=2), n_estimators=10).fit(train_X, train_y)

rfc.feature_importances_

In [ ]:
# Deep Learning Model

from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

from keras.models import Sequential
model = Sequential()

from keras.layers import Activation, Dense, Dropout, BatchNormalization
model.add(Dense(len(train_X.columns), input_shape = (len(train_X),)))    # 입력층 
model.add(Activation('relu'))
model.add(Dense(128))                         # 은닉층 1
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(128))                         # 은닉층 2
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1))                          # 출력층 => output dimension == 1 (regression problem)
model.add(Activation('relu'))             

model.summary()

model.compile(optimizer='rmsprop',
              loss='mse', epoch=200, batch_size=8,
              metrics=['accuracy'])

model.fit(train_X, train_y)

In [ ]:
# RESNET Layer-50, 이미지 분류

from keras import models, layers
from keras import Input
from keras.models import Model, load_model

from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add

import numpy as np
import math

# number of classes
K = 2

# Input Shape
input_tensor = Input(shape=(120, 120, 3), dtype='float32', name='input')

def conv1_layer(x):    
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)
 
    return x   

def conv2_layer(x):         
    x = MaxPooling2D((3, 3), 2)(x)     
 
    shortcut = x
 
    for i in range(3):
        if (i == 0):
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(shortcut)            
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut])
            x = Activation('relu')(x)
            
            shortcut = x
 
        else:
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])   
            x = Activation('relu')(x)  
 
            shortcut = x        
    
    return x

def conv3_layer(x):        
    shortcut = x    
    
    for i in range(4):     
        if(i == 0):            
            x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)    
 
            shortcut = x              
        
        else:
            x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])     
            x = Activation('relu')(x)
 
            shortcut = x      
            
    return x

def conv4_layer(x):
    shortcut = x        
  
    for i in range(6):     
        if(i == 0):            
            x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(1024, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)
 
            shortcut = x               
        
        else:
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)
 
            shortcut = x      
 
    return x

def conv5_layer(x):
    shortcut = x    
  
    for i in range(3):     
        if(i == 0):            
            x = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(2048, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])  
            x = Activation('relu')(x)      
 
            shortcut = x               
        
        else:
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)           
            
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)       
 
            shortcut = x                  
 
    return x

x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
x = conv5_layer(x)

x = GlobalAveragePooling2D()(x)
output_tensor = Dense(K, activation='softmax')(x)

resnet50 = Model(input_tensor, output_tensor)
resnet50.summary()

# 컴파일
resnet50.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

resnet50.fit(X_train, y_train, epochs=10,verbose=1)


In [1]:
# DCGAN 이미지 생성 (Cycle GAN?)



In [ ]:
# LSTM 시계열 분석

In [ ]:
# Object Detection

